In [1]:
from init import *
from helpers_analyses import *


Getting english data
Getting french data
Getting emotion words and bird names
number of overlapping words between Leuven and Rosch: 25
correlation between Leuven and Rosch ratings: (0.9519771567894879, 2.5885716906113406e-13)


In [2]:
# reconstruction in other categories
stats = []

other_cats = read_norms_1980(norms_1980_path)
other_cats = {c: {w: -p for w, p in v.items()} for c, v in other_cats.items()}
for cat, cat_proto in other_cats.items():
    cat_words = list(cat_proto.keys())
    seeds = sorted(cat_words, key=lambda w: cat_proto[w], reverse=True)
    p_w, e_w = estimate_proto(seeds, cat_words, data, cat_proto)
    r, p = pearsonr(p_w, e_w)
    stats.append({
        'category': cat,
        'r': '%.3g' % r,
        'p-value': '%.3g' % p,
        'n': len(p_w)
    })

stats_df = pd.DataFrame(stats)
stats_df = stats_df.reindex(['category', 'r', 'p-value', 'n'], axis=1)
print(stats_df.to_latex(index=False, column_format='||c | c | c | c||'))


seeds: ['pants', 'shirt', 'dress', 'trousers', 'blouse']
seeds: ['chair', 'table', 'couch', 'sofa', 'desk']
seeds: ['dove', 'robin', 'sparrow', 'pigeon', 'bluebird']
seeds: ['saw', 'hammer', 'screwdriver', 'nails', 'drill']
seeds: ['red', 'blue', 'yellow', 'green', 'orange']
seeds: ['france', 'england', 'italy', 'japan', 'germany']
seeds: ['dog', 'cat', 'horse', 'cow', 'wolf']
seeds: ['orange', 'apple', 'pear', 'banana', 'peach']
seeds: ['cotton', 'wool', 'silk', 'linen', 'flannel']
seeds: ['dollars', 'dimes', 'pennies', 'quarters', 'nickels']
seeds: ['knife', 'spoon', 'fork', 'pot', 'spatula']
seeds: ['priest', 'minister', 'reverend', 'clergyman', 'bishop']
seeds: ['iron', 'steel', 'tin', 'aluminum', 'copper']
seeds: ['piano', 'guitar', 'flute', 'trumpet', 'violin']
seeds: ['mountain', 'volcano', 'island', 'valley', 'cave']
seeds: ['milk', 'water', 'lemonade', 'tea', 'coffee']
seeds: ['roof', 'door', 'wall', 'room', 'floor']
seeds: ['head', 'legs', 'hand', 'foot', 'arms']
seeds: ['bas

In [3]:
# precompute semantic change in other categories
k = 100
t1, t2 = 9, -1
pos = NOUN

other_cats = read_norms_1980(norms_1980_path)
measures_other_cat = {}
for cat, cat_proto in other_cats.items():
    cat_words = list(cat_proto.keys())
    measures_other_cat[cat] = nn_measure_over_words(cat_words, pos, pos_data, data, t1, t2, k)


In [4]:
# regression for other categories
from scipy.stats import binom_test

t_to_use = 1890
other_cats_rev = {c: {w: -p for w, p in v.items()} for c, v in other_cats.items()}

stats_R_sqr = []
stats_coef = []
stats_pcor = []
num_sig, num_total = 0, 0
for cat, cat_proto in other_cats_rev.items():
    print(cat)
    measure_other_cat = measures_other_cat[cat]
    cat_words = list(measure_other_cat.keys())
    freqs = {w: np.log(freqs_all[w][t_to_use]) for w in measure_other_cat}
    result, n, pr, pp, pr2, pp2 = regression_results(cat_words, measure_other_cat, cat_proto, freqs)
    stats_R_sqr.append({
        'category': cat,
        'adjusted $R^2$': '%.3g' % result.rsquared_adj,
        'p-value': '%.3g' % result.f_pvalue,
        'n': len(cat_words)
    })
    stats_coef.append({
        'category': cat,
        'proto': '%.3g' % result.params[1],
        'p-value proto': '%.3g' % result.pvalues[1],
        'frequency': '%.3g' % result.params[2],
        'p-value frequency': '%.3g' % result.pvalues[2],
        'length': '%.3g' % result.params[3],
        'p-value length': '%.3g' % result.pvalues[3],
        'n': len(cat_words)
    })
    stats_pcor.append({
        'category': cat,
        'partial correlation': '%.3g' % pr,
        'p-value': '%.3g' % pp,
        'n': len(cat_words)
    })
    num_sig += 1 if pr < 0 and pp < 0.05 else 0
    num_total += 1

print('Binomial test', binom_test(num_sig, n=num_total, p=0.5), 'x=', num_sig)
    
stats_df = pd.DataFrame(stats_R_sqr)
stats_df = stats_df.reindex(['category', 'adjusted $R^2$', 'p-value', 'n'], axis=1)
print(stats_df.to_latex(index=False, column_format='||c | c | c | c||'))

stats_df = pd.DataFrame(stats_coef)
stats_df = stats_df.reindex(['category', 'proto', 'p-value proto', 'frequency', 'p-value frequency', 'length', 'p-value length', 'n'], axis=1)
print(stats_df.to_latex(index=False, column_format='||c | c | c | c | c | c | c | c||'))

stats_df = pd.DataFrame(stats_pcor)
stats_df = stats_df.reindex(['category', 'partial correlation', 'p-value', 'n'], axis=1)
print(stats_df.to_latex(index=False, column_format='||c | c | c | c||'))



Article of clothing
                            OLS Regression Results                            
Dep. Variable:                 change   R-squared:                       0.214
Model:                            OLS   Adj. R-squared:                  0.075
Method:                 Least Squares   F-statistic:                     1.544
Date:                Tue, 09 Jun 2020   Prob (F-statistic):              0.239
Time:                        01:27:08   Log-Likelihood:                 30.002
No. Observations:                  21   AIC:                            -52.00
Df Residuals:                      17   BIC:                            -47.83
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept           0.

/Users/johnxu/anaconda3/lib/python3.7/site-packages/scipy/stats/stats.py:1450: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=18
  "anyway, n=%i" % int(n))
/Users/johnxu/anaconda3/lib/python3.7/site-packages/scipy/stats/stats.py:1450: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=15
  "anyway, n=%i" % int(n))
/Users/johnxu/anaconda3/lib/python3.7/site-packages/scipy/stats/stats.py:1450: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=17
  "anyway, n=%i" % int(n))
/Users/johnxu/anaconda3/lib/python3.7/site-packages/scipy/stats/stats.py:1450: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=16
  "anyway, n=%i" % int(n))


                            OLS Regression Results                            
Dep. Variable:                 change   R-squared:                       0.095
Model:                            OLS   Adj. R-squared:                 -0.099
Method:                 Least Squares   F-statistic:                    0.4875
Date:                Tue, 09 Jun 2020   Prob (F-statistic):              0.696
Time:                        01:27:09   Log-Likelihood:                 26.625
No. Observations:                  18   AIC:                            -45.25
Df Residuals:                      14   BIC:                            -41.69
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept           0.8336      0.130     

                            OLS Regression Results                            
Dep. Variable:                 change   R-squared:                       0.479
Model:                            OLS   Adj. R-squared:                  0.408
Method:                 Least Squares   F-statistic:                     6.735
Date:                Tue, 09 Jun 2020   Prob (F-statistic):            0.00216
Time:                        01:27:09   Log-Likelihood:                 38.243
No. Observations:                  26   AIC:                            -68.49
Df Residuals:                      22   BIC:                            -63.45
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept           0.4367      0.108     

pearson  27 -0.315  [-0.62, 0.07]  0.099   0.024  0.10951  0.807  0.368
partial correlation, change and frequency given prototypicality
          n      r          CI95%     r2  adj_r2         p-val       BF10  \
pearson  27 -0.789  [-0.9, -0.58]  0.623   0.592  9.832843e-07  1.993e+04   

         power  
pearson    1.0  
Nonalcoholic beverage
                            OLS Regression Results                            
Dep. Variable:                 change   R-squared:                       0.641
Model:                            OLS   Adj. R-squared:                  0.534
Method:                 Least Squares   F-statistic:                     5.962
Date:                Tue, 09 Jun 2020   Prob (F-statistic):             0.0134
Time:                        01:27:09   Log-Likelihood:                 21.614
No. Observations:                  14   AIC:                            -35.23
Df Residuals:                      10   BIC:                            -32.67
Df Model:            

/Users/johnxu/anaconda3/lib/python3.7/site-packages/scipy/stats/stats.py:1450: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=14
  "anyway, n=%i" % int(n))


                            OLS Regression Results                            
Dep. Variable:                 change   R-squared:                       0.460
Model:                            OLS   Adj. R-squared:                  0.344
Method:                 Least Squares   F-statistic:                     3.972
Date:                Tue, 09 Jun 2020   Prob (F-statistic):             0.0306
Time:                        01:27:09   Log-Likelihood:                 32.817
No. Observations:                  18   AIC:                            -57.63
Df Residuals:                      14   BIC:                            -54.07
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept           0.4939      0.131     

                            OLS Regression Results                            
Dep. Variable:                 change   R-squared:                       0.316
Model:                            OLS   Adj. R-squared:                  0.195
Method:                 Least Squares   F-statistic:                     2.619
Date:                Tue, 09 Jun 2020   Prob (F-statistic):             0.0844
Time:                        01:27:10   Log-Likelihood:                 29.147
No. Observations:                  21   AIC:                            -50.29
Df Residuals:                      17   BIC:                            -46.12
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept           0.6258      0.133     

/Users/johnxu/anaconda3/lib/python3.7/site-packages/scipy/stats/stats.py:1450: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=19
  "anyway, n=%i" % int(n))
